# Results inspection
Takes the results from the 5 sub-sample validations and averages + computes the standard deviation of the predictions.

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
from collections import defaultdict
from typing import Literal
import os

model_types = {
    "single": "singlemodel",
    "token": "tokenmodel",
}

model_type = model_types["single"]
model_type = model_types["token"]
print(model_type)

# fetch all the related folders (dataset_{single/token}model.N)
datasets = {
    "europarl": "europarl",
    "opensub": "opensub",
    "tatoeba": "tatoeba",
}

dataset = datasets["europarl"]
# dataset = datasets["opensub"]

folder_id = f"{dataset}_{model_type}"
folders = [f for f in os.listdir(os.path.join("..", "results", dataset)) if dataset in f]
folders = sorted(folders, key=lambda x: int(x.split(".")[-1]))
print(folders)
def get_combined(_type: Literal["metrics", "translations"]="metrics"):
    combined = {}
    for folder in folders:
        if model_type not in folder:
            continue
        print(f"Fetching from {folder}")
        folder = os.path.join("..", "results", dataset, folder)
        for files in sorted(os.listdir(folder)):
            if _type not in files:
                continue
            lang = files.split(".")[0]
            if lang not in combined:
                combined[lang] = {}
            
            with open(f"{folder}/{files}") as f:
                data = json.load(f)
                # baseline, 0.5, 0.6, ..., 1.0

                if _type == "metrics":
                    for model_key, metric_values in data.items():
                        if model_key not in combined[lang]:
                            combined[lang][model_key] = defaultdict(list)

                        for metric_key, metric_value in metric_values.items():
                            combined[lang][model_key][metric_key].append(metric_value)
                if _type == "translations":
                    for model_key, translation_values in data.items():
                        combined[lang][model_key] = translation_values
    return combined

all_metrics = get_combined("metrics")
all_transl = get_combined("translations")

tokenmodel
['europarl_tokenmodel.0', 'europarl_singlemodel.0', 'europarl_singlemodel.1', 'europarl_tokenmodel.1', 'europarl_singlemodel.2', 'europarl_tokenmodel.2', 'europarl_singlemodel.3', 'europarl_tokenmodel.3', 'europarl_singlemodel.4', 'europarl_tokenmodel.4']
Fetching from europarl_tokenmodel.0
Fetching from europarl_tokenmodel.1
Fetching from europarl_tokenmodel.2
Fetching from europarl_tokenmodel.3
Fetching from europarl_tokenmodel.4
Fetching from europarl_tokenmodel.0
Fetching from europarl_tokenmodel.1
Fetching from europarl_tokenmodel.2
Fetching from europarl_tokenmodel.3
Fetching from europarl_tokenmodel.4


In [8]:
import pandas as pd
import numpy as np
# calc the mean + std for each list of the N metric values
import sys
sys.path.append("../")
from evaluation_tools import print_metrics

ADD_STD = False

for ADD_STD in [True, False]:
    model_types_to_visualize = ["baseline", "0.5", "0.7", "1.0"]
    ignored_metrics = ["chrf++", "rl",]

    presentable_metrics = {lang: {} for lang in all_metrics.keys()}
    for lang, metrics in all_metrics.items():
        for model_type, model_values in metrics.items():
            # model type is the compressoin level
            if model_type not in model_types_to_visualize:
                continue
            # print(model_type)
            presentable_metrics[lang][model_type] = {}
            for metrics, metric_values in model_values.items():
                if metrics in ignored_metrics:
                    continue
                values = np.array(metric_values)
                mean = np.mean(values)
                std = np.std(values)
                std_as_percentage = std / mean * 100
                #value_str = f"{mean:.2f} ± {std:.3f}"
                if ADD_STD:
                    value_str = f"{mean:.2f} ({std:.2f})"
                    #presentable_metrics[lang][model_type][metrics] = (mean.round(2), std.round(3))
                    presentable_metrics[lang][model_type][metrics] = value_str
                else:
                    presentable_metrics[lang][model_type][metrics] = float(mean.round(2))
                    

    dataframes = {}

    for lang, metrics in presentable_metrics.items():
        dataframes[lang] = pd.DataFrame(metrics)

    for lang in dataframes.keys():
        print_metrics(_lang=lang, _metrics=dataframes[lang], std=ADD_STD, transpose=False)


,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio
baseline,39.75 (0.45),0.67 (0.00),0.50 (0.00),66.67 (0.38),0.64 (0.00),0.88 (0.00),1.20 (0.06)
0.5,33.28 (0.51),0.60 (0.00),0.43 (0.00),57.65 (0.29),0.52 (0.00),0.86 (0.00),0.80 (0.01)
0.7,37.91 (0.51),0.65 (0.01),0.47 (0.00),62.55 (0.33),0.59 (0.00),0.87 (0.00),0.93 (0.00)
1.0,39.75 (0.63),0.67 (0.00),0.49 (0.00),64.55 (0.48),0.62 (0.00),0.88 (0.00),1.01 (0.00)



\begin{tabular}{llllllll}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio \\
\midrule
baseline & 39.75 (0.45) & 0.67 (0.00) & 0.50 (0.00) & 66.67 (0.38) & 0.64 (0.00) & 0.88 (0.00) & 1.20 (0.06) \\
0.5 & 33.28 (0.51) & 0.60 (0.00) & 0.43 (0.00) & 57.65 (0.29) & 0.52 (0.00) & 0.86 (0.00) & 0.80 (0.01) \\
0.7 & 37.91 (0.51) & 0.65 (0.01) & 0.47 (0.00) & 62.55 (0.33) & 0.59 (0.00) & 0.87 (0.00) & 0.93 (0.00) \\
1.0 & 39.75 (0.63) & 0.67 (0.00) & 0.49 (0.00) & 64.55 (0.48) & 0.62 (0.00) & 0.88 (0.00) & 1.01 (0.00) \\
\bottomrule
\end{tabular}



,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio
baseline,34.12 (0.98),0.69 (0.01),0.50 (0.00),64.60 (0.54),0.61 (0.01),0.88 (0.00),1.05 (0.01)
0.5,18.66 (0.58),0.52 (0.00),0.36 (0.00),43.95 (0.32),0.39 (0.01),0.82 (0.00),0.60 (0.01)
0.7,28.25 (0.77),0.64 (0.00),0.45 (0.00),57.15 (0.43),0.53 (0.01),0.86 (0.00),0.84 (0.01)
1.0,32.42 (0.92),0.68 (0.00),0.49 (0.00),62.04 (0.45),0.59 (0.01),0.88 (0.00),0.96 (0.00)



\begin{tabular}{llllllll}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio \\
\midrule
baseline & 34.12 (0.98) & 0.69 (0.01) & 0.50 (0.00) & 64.60 (0.54) & 0.61 (0.01) & 0.88 (0.00) & 1.05 (0.01) \\
0.5 & 18.66 (0.58) & 0.52 (0.00) & 0.36 (0.00) & 43.95 (0.32) & 0.39 (0.01) & 0.82 (0.00) & 0.60 (0.01) \\
0.7 & 28.25 (0.77) & 0.64 (0.00) & 0.45 (0.00) & 57.15 (0.43) & 0.53 (0.01) & 0.86 (0.00) & 0.84 (0.01) \\
1.0 & 32.42 (0.92) & 0.68 (0.00) & 0.49 (0.00) & 62.04 (0.45) & 0.59 (0.01) & 0.88 (0.00) & 0.96 (0.00) \\
\bottomrule
\end{tabular}



,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio
baseline,33.67 (0.91),0.62 (0.01),0.42 (0.01),65.13 (0.56),0.60 (0.01),0.87 (0.00),1.06 (0.02)
0.5,26.54 (0.87),0.53 (0.01),0.33 (0.01),55.62 (0.61),0.47 (0.01),0.84 (0.00),0.81 (0.01)
0.7,29.83 (0.87),0.58 (0.01),0.37 (0.01),59.82 (0.59),0.54 (0.01),0.86 (0.00),0.93 (0.01)
1.0,31.26 (0.82),0.60 (0.01),0.39 (0.01),61.61 (0.60),0.56 (0.01),0.86 (0.00),0.99 (0.01)


...


\begin{tabular}{lrrrrrrrr}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio & comp_score \\
\midrule
baseline & 34.12 & 0.69 & 0.50 & 64.60 & 0.61 & 0.88 & 1.05 & 58 \\
0.5 & 18.66 & 0.52 & 0.36 & 43.95 & 0.39 & 0.82 & 0.60 & 64 \\
0.7 & 28.25 & 0.64 & 0.45 & 57.15 & 0.53 & 0.86 & 0.84 & 62 \\
1.0 & 32.42 & 0.68 & 0.49 & 62.04 & 0.59 & 0.88 & 0.96 & 60 \\
\bottomrule
\end{tabular}



,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio,comp_score
baseline,33.67,0.62,0.42,65.13,0.60,0.87,1.06,65
0.5,26.54,0.53,0.33,55.62,0.47,0.84,0.81,67
0.7,29.83,0.58,0.37,59.82,0.54,0.86,0.93,66
1.0,31.26,0.60,0.39,61.61,0.56,0.86,0.99,65



\begin{tabular}{lrrrrrrrr}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio & comp_score \\
\midrule
baseline & 33.67 & 0.62 & 0.42 & 65.13 & 0.60 & 0.87 & 1.06 & 65 \\
0.5 & 26.54 & 0.53 & 0.33 & 55.62 & 0.47 & 0.84 & 0.81 & 67 \\
0.7 & 29.83 & 0.58 & 0.37 & 59.82 & 0.54 & 0.86 & 0.93 & 66 \\
1.0 & 31.26 & 0.60 & 0.39 & 61.61 & 0.56 & 0.86 & 0.99 & 65 \\
\bottomrule
\end{tabular}



,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio,comp_score
baseline,12.28,0.44,0.24,48.84,0.44,0.80,1.51,36
0.5,21.37,0.50,0.29,48.79,0.43,0.83,0.77,65
0.7,23.93,0.54,0.33,52.52,0.49,0.85,0.88,63
1.0,26.05,0.56,0.34,55.11,0.52,0.86,0.95,62



\begin{tabular}{lrrrrrrrr}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio & comp_score \\
\midrule
baseline & 12.28 & 0.44 & 0.24 & 48.84 & 0.44 & 0.80 & 1.51 & 36 \\
0.5 & 21.37 & 0.50 & 0.29 & 48.79 & 0.43 & 0.83 & 0.77 & 65 \\
0.7 & 23.93 & 0.54 & 0.33 & 52.52 & 0.49 & 0.85 & 0.88 & 63 \\
1.0 & 26.05 & 0.56 & 0.34 & 55.11 & 0.52 & 0.86 & 0.95 & 62 \\
\bottomrule
\end{tabular}



,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio
baseline,39.75 (0.45),0.67 (0.00),0.50 (0.00),66.67 (0.38),0.64 (0.00),0.88 (0.00),1.20 (0.06)
0.5,33.28 (0.51),0.60 (0.00),0.43 (0.00),57.65 (0.29),0.52 (0.00),0.86 (0.00),0.80 (0.01)
0.7,37.91 (0.51),0.65 (0.01),0.47 (0.00),62.55 (0.33),0.59 (0.00),0.87 (0.00),0.93 (0.00)
1.0,39.75 (0.63),0.67 (0.00),0.49 (0.00),64.55 (0.48),0.62 (0.00),0.88 (0.00),1.01 (0.00)



\begin{tabular}{llllllll}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio \\
\midrule
baseline & 39.75 (0.45) & 0.67 (0.00) & 0.50 (0.00) & 66.67 (0.38) & 0.64 (0.00) & 0.88 (0.00) & 1.20 (0.06) \\
0.5 & 33.28 (0.51) & 0.60 (0.00) & 0.43 (0.00) & 57.65 (0.29) & 0.52 (0.00) & 0.86 (0.00) & 0.80 (0.01) \\
0.7 & 37.91 (0.51) & 0.65 (0.01) & 0.47 (0.00) & 62.55 (0.33) & 0.59 (0.00) & 0.87 (0.00) & 0.93 (0.00) \\
1.0 & 39.75 (0.63) & 0.67 (0.00) & 0.49 (0.00) & 64.55 (0.48) & 0.62 (0.00) & 0.88 (0.00) & 1.01 (0.00) \\
\bottomrule
\end{tabular}



,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio
baseline,34.12 (0.98),0.69 (0.01),0.50 (0.00),64.60 (0.54),0.61 (0.01),0.88 (0.00),1.05 (0.01)
0.5,18.66 (0.58),0.52 (0.00),0.36 (0.00),43.95 (0.32),0.39 (0.01),0.82 (0.00),0.60 (0.01)
0.7,28.25 (0.77),0.64 (0.00),0.45 (0.00),57.15 (0.43),0.53 (0.01),0.86 (0.00),0.84 (0.01)
1.0,32.42 (0.92),0.68 (0.00),0.49 (0.00),62.04 (0.45),0.59 (0.01),0.88 (0.00),0.96 (0.00)



\begin{tabular}{llllllll}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio \\
\midrule
baseline & 34.12 (0.98) & 0.69 (0.01) & 0.50 (0.00) & 64.60 (0.54) & 0.61 (0.01) & 0.88 (0.00) & 1.05 (0.01) \\
0.5 & 18.66 (0.58) & 0.52 (0.00) & 0.36 (0.00) & 43.95 (0.32) & 0.39 (0.01) & 0.82 (0.00) & 0.60 (0.01) \\
0.7 & 28.25 (0.77) & 0.64 (0.00) & 0.45 (0.00) & 57.15 (0.43) & 0.53 (0.01) & 0.86 (0.00) & 0.84 (0.01) \\
1.0 & 32.42 (0.92) & 0.68 (0.00) & 0.49 (0.00) & 62.04 (0.45) & 0.59 (0.01) & 0.88 (0.00) & 0.96 (0.00) \\
\bottomrule
\end{tabular}



,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio
baseline,33.67 (0.91),0.62 (0.01),0.42 (0.01),65.13 (0.56),0.60 (0.01),0.87 (0.00),1.06 (0.02)
0.5,26.54 (0.87),0.53 (0.01),0.33 (0.01),55.62 (0.61),0.47 (0.01),0.84 (0.00),0.81 (0.01)
0.7,29.83 (0.87),0.58 (0.01),0.37 (0.01),59.82 (0.59),0.54 (0.01),0.86 (0.00),0.93 (0.01)
1.0,31.26 (0.82),0.60 (0.01),0.39 (0.01),61.61 (0.60),0.56 (0.01),0.86 (0.00),0.99 (0.01)



\begin{tabular}{llllllll}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio \\
\midrule
baseline & 33.67 (0.91) & 0.62 (0.01) & 0.42 (0.01) & 65.13 (0.56) & 0.60 (0.01) & 0.87 (0.00) & 1.06 (0.02) \\
0.5 & 26.54 (0.87) & 0.53 (0.01) & 0.33 (0.01) & 55.62 (0.61) & 0.47 (0.01) & 0.84 (0.00) & 0.81 (0.01) \\
0.7 & 29.83 (0.87) & 0.58 (0.01) & 0.37 (0.01) & 59.82 (0.59) & 0.54 (0.01) & 0.86 (0.00) & 0.93 (0.01) \\
1.0 & 31.26 (0.82) & 0.60 (0.01) & 0.39 (0.01) & 61.61 (0.60) & 0.56 (0.01) & 0.86 (0.00) & 0.99 (0.01) \\
\bottomrule
\end{tabular}



,bleu,r1,r2,chrF,meteor,bert_f1,len_ratio
baseline,12.28 (0.66),0.44 (0.00),0.24 (0.00),48.84 (0.71),0.44 (0.00),0.80 (0.00),1.51 (0.09)
0.5,21.37 (0.52),0.50 (0.00),0.29 (0.01),48.79 (0.55),0.43 (0.00),0.83 (0.00),0.77 (0.01)
0.7,23.93 (0.58),0.54 (0.00),0.33 (0.00),52.52 (0.42),0.49 (0.00),0.85 (0.00),0.88 (0.00)
1.0,26.05 (0.38),0.56 (0.00),0.34 (0.00),55.11 (0.35),0.52 (0.01),0.86 (0.00),0.95 (0.00)



\begin{tabular}{llllllll}
\toprule
 & bleu & r1 & r2 & chrF & meteor & bert_f1 & len_ratio \\
\midrule
baseline & 12.28 (0.66) & 0.44 (0.00) & 0.24 (0.00) & 48.84 (0.71) & 0.44 (0.00) & 0.80 (0.00) & 1.51 (0.09) \\
0.5 & 21.37 (0.52) & 0.50 (0.00) & 0.29 (0.01) & 48.79 (0.55) & 0.43 (0.00) & 0.83 (0.00) & 0.77 (0.01) \\
0.7 & 23.93 (0.58) & 0.54 (0.00) & 0.33 (0.00) & 52.52 (0.42) & 0.49 (0.00) & 0.85 (0.00) & 0.88 (0.00) \\
1.0 & 26.05 (0.38) & 0.56 (0.00) & 0.34 (0.00) & 55.11 (0.35) & 0.52 (0.01) & 0.86 (0.00) & 0.95 (0.00) \\
\bottomrule
\end{tabular}



In [ ]:
dataframes.keys()

In [ ]:
first_entry = list(all_transl.keys())[0]
translation_keys = all_transl[first_entry].keys()
print(translation_keys)

# Good IDs

## norwegian nob
```
index: 90
source
>>nob<< He is not flesh but spirit. The light of eternal mind.

target
Sinnets evige lys.

baseline
Han er ikke kjøtt, men ånden, lyset til det evige sinnet.

0.5
Det evige sinns lys.

0.6
Det evige sinns lys.

0.7
Det evige sinns lys.

0.8
Han er ikke kjøtt, men ånd, evig sinds lys.

0.9
Han er ikke kjøtt, men ånd, evig sinds lys.

1.0
Han er ikke kjøtt, men ånd, evig sinds lys.

___
index: 279
source
>>nob<< Easy, easy, easy, easy.

target
Rolig...

baseline
Lett, enkelt, enkelt.

0.5
Rolig.
___
index: 843
source
>>nob<< But when he carried you over the threshold, Karl, and he... He laid you down and gently kissed you, didn't he, didn't he say any...

target
Han bar deg over terskelen og kysset deg.

baseline
Men da han bar deg over terskelen, Karl, og han... han la deg ned og forsiktig kysset deg, ikke sant, sa han ikke...

0.5
Men da han bar deg over terskelen, la han deg ned og kysset deg, ikke sant?

0.6
Men da han bar deg over terskelen, la han deg ned og kysset deg forsiktig, sa han ikke...
___


index: 355
source
>>nob<< All right, well...

target
Greit.

baseline
Ok, vel...

0.5
Greit.

0.6
Ok, ok...

0.7
Greit, ja vel...

___


```




In [ ]:
import random
def peek(lang: str):
    data = all_transl[lang]
    index = random.randint(0, len(data["source"]) - 1)
    print("index:", index)

    sample = {}
    for key in translation_keys:
        print(key)
        print(data[key][index])
        print()
        sample[key] = data[key][index]
    
    #return sample

lang = "no"
peek(lang)